## Data Merging and Cleaning


In [4]:
## importing pandas
import pandas as pd

In [5]:
#using this block to read csv files and dropping out the the columns from the file that the code will not be making use of.
translation = pd.read_csv("product_category_name_translation.csv")
translation.head()

order_items = pd.read_csv("olist_order_items_dataset.csv")
order_items = order_items.drop(['shipping_limit_date', 'seller_id'], axis=1)
order_items.head()

products = pd.read_csv("olist_products_dataset.csv")
products = products.drop(['product_description_lenght','product_photos_qty', 'product_weight_g', 
                             'product_length_cm', 'product_height_cm', 
                             'product_width_cm'], axis=1)
products.head()

orders = pd.read_csv("olist_orders_dataset.csv")
orders = orders.drop(['order_status', 'order_approved_at', 'order_delivered_carrier_date', 
                      'order_delivered_customer_date', 'order_estimated_delivery_date'], axis=1)
orders.head()

customers = pd.read_csv("olist_customers_dataset.csv")
customers = customers.drop(['customer_zip_code_prefix', 'customer_city', 'customer_state'], axis=1)
customers.head()

,customer_id,customer_unique_id
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066


In [6]:
##merging the different csv files that will be used in the association rule algorithm.
merged = pd.merge(translation, products, on='product_category_name', how='inner')
merged = pd.merge(merged, order_items, on = 'product_id', how='inner')
merged = pd.merge(orders, merged, on='order_id', how='inner')
merged = pd.merge(customers, merged, on='customer_id', how='inner')

merged = merged.drop(['customer_id', 'product_category_name'], axis=1)
merged.head()

,customer_unique_id,order_id,order_purchase_timestamp,product_category_name_english,product_id,product_name_lenght,order_item_id,price,freight_value
0,861eff4711a542e4b93843c6dd7febb0,00e7ee1b050b8499577073aeb2a297a1,2017-05-16 15:05:35,office_furniture,a9516a079e37a9c9c36b9b78b10169e8,41.0,1,124.99,21.88
1,290c77bc529b7ac935b93aa66c333dc3,29150127e6685892b6eab3eec79f59c7,2018-01-12 20:48:24,housewares,4aa6014eceb682077f9dc4bffebc05b0,43.0,1,289.00,46.48
2,060e732b5b29e8181a18229c7b0b2b5e,b2059ed67ce144a36e2aa97d2c9e9ad2,2018-05-19 16:07:45,office_furniture,bd07b66896d6f1494f5b86251848ced7,55.0,1,139.94,17.79
3,259dac757896d24d7702b9acbbff3f3c,951670f92359f4fe4a63112aa7306eba,2018-03-13 16:06:38,office_furniture,a5647c44af977b148e0a3a4751a09e2e,48.0,1,149.94,23.36
4,345ecd01c38d18a9036ed96c73b8d066,6b7d50bd145f6fc7f33cebabd7e49d0f,2018-07-29 09:51:30,home_confort,9391a573abe00141c56e38d84d7d5b3b,61.0,1,230.00,22.25


In [7]:
##filtered out the merged CSV file. All of these steps are taken as part of data sorting, and cleaing.
##we gorup orders that are in the same shopping cart using order_purchase_timestamp and unique_user_id
##we remove shopping carts that have only one category of items
filtered_merged = merged.sort_values(by=['customer_unique_id', 'order_purchase_timestamp'])
filtered_merged = filtered_merged.reset_index()
filtered_merged=filtered_merged[filtered_merged.duplicated(['customer_unique_id', 'order_purchase_timestamp'], keep=False)]
filtered_merged = filtered_merged.drop_duplicates(['customer_unique_id', 'product_category_name_english'], keep='last')
filtered_merged = filtered_merged=filtered_merged[filtered_merged.duplicated(['customer_unique_id'], keep=False)]

In [8]:
basic_merged = filtered_merged
basic_merged = basic_merged.drop(['product_name_lenght', 'order_item_id', 'price', 
                   'freight_value', 'product_id',
                   'order_purchase_timestamp', 'order_id', 'index'], axis=1)

In [9]:
##dropping out the nan values.
basic_merged = basic_merged.dropna()
lst = list(basic_merged.groupby('customer_unique_id')['product_category_name_english'].apply(list))

## Data Visualization

In [10]:
##importing all the external libraries that we would be making use of.
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import networkx as nx
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori,association_rules
import matplotlib.pyplot as plt
plt.style.use('default')

In [27]:
# 1. Gathering All Items of Each Transactions into a Numpy Array
from itertools import chain
lst_squashed = list(chain.from_iterable(lst))
transaction = lst_squashed
transaction = np.array(transaction)

# 2. Transforming Them into Pandas DataFrame
df = pd.DataFrame(transaction, columns=["items"]) 
df["incident_count"] = 1 # Put 1 to Each Item For Making Countable Table, to be able to perform Group By

# 3. Deleting NaN Items from Dataset
indexNames = df[df['items'] == "nan" ].index
df.drop(indexNames , inplace=True)

# 4. Making  a New Appropriate Pandas DataFrame for Visualizations  
df_table = df.groupby("items").sum().sort_values("incident_count", ascending=False).reset_index()

# 5. Initial Visualizations
df_table.head(10).style.background_gradient(cmap='Blues')


,items,incident_count
0,furniture_decor,246
1,bed_bath_table,230
2,housewares,115
3,baby,100
4,garden_tools,83
5,sports_leisure,81
6,health_beauty,77
7,cool_stuff,75
8,computers_accessories,58
9,toys,56


In [28]:
##drawing a tree map of 
##df_table["all"] = "all" # to have a same origin

#fig = px.treemap(df_table.head(30), path=['all', "items"], values='incident_count',
  #                color=df_table["incident_count"].head(30), hover_data=['items'],
   #               color_continuous_scale='Blues',
           #       )
#fig.show()

In [13]:
# Transform Every Transaction to Seperate List & Gather Them into Numpy Array
# By Doing So, We Will Be Able To Iterate Through Array of Transactions

transaction = lst_squashed
transaction = np.array(transaction)

# Create a DataFrame In Order To Check Status of Top20 Items

top20 = df_table["items"].head(20).values
array = []
df_top20_multiple_record_check = pd.DataFrame(columns=top20)

for i in range(0, len(top20)):
    array = []
    for j in range(0,transaction.shape[0]):
        array.append(np.count_nonzero(transaction[j]==top20[i]))
        if len(array) == len(lst_squashed):
            df_top20_multiple_record_check[top20[i]] = array
        else:
            continue
            

df_top20_multiple_record_check.head(10)

,furniture_decor,bed_bath_table,housewares,baby,garden_tools,sports_leisure,health_beauty,cool_stuff,computers_accessories,toys,home_confort,watches_gifts,auto,perfumery,stationery,telephony,home_construction,electronics,fashion_bags_accessories,construction_tools_construction
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
df_top20_multiple_record_check.describe()

,furniture_decor,bed_bath_table,housewares,baby,garden_tools,sports_leisure,health_beauty,cool_stuff,computers_accessories,toys,home_confort,watches_gifts,auto,perfumery,stationery,telephony,home_construction,electronics,fashion_bags_accessories,construction_tools_construction
count,1709.000000,1709.000000,1709.000000,1709.000000,1709.000000,1709.000000,1709.000000,1709.000000,1709.000000,1709.000000,1709.000000,1709.000000,1709.000000,1709.000000,1709.000000,1709.000000,1709.000000,1709.000000,1709.000000,1709.000000
mean,0.143944,0.134582,0.067291,0.058514,0.048566,0.047396,0.045056,0.043885,0.033938,0.032768,0.029842,0.028672,0.021650,0.019310,0.019310,0.018139,0.016384,0.015799,0.012873,0.012288
std,0.351136,0.341376,0.250598,0.234781,0.215023,0.212547,0.207487,0.204900,0.181123,0.178080,0.170201,0.166931,0.145581,0.137651,0.137651,0.133494,0.126984,0.124733,0.112760,0.110200
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [15]:
# 1. Gather Only First Choice of Each Transactions into Numpy Array
# Similar Pattern to Above, Only Change is the Column Number "0" in Append Function
transaction = lst_squashed
transaction = np.array(transaction)

# 2. Transform Them a Pandas DataFrame
df_first = pd.DataFrame(transaction, columns=["items"])
df_first["incident_count"] = 1

# 3. Delete NaN Items from Dataset
indexNames = df_first[df_first['items'] == "nan" ].index
df_first.drop(indexNames , inplace=True)

# 4. Final Step: Make a New Appropriate Pandas DataFrame for Visualizations  
df_table_first = df_first.groupby("items").sum().sort_values("incident_count", ascending=False).reset_index()
df_table_first["food"] = "food"
df_table_first = df_table_first.truncate(before=-1, after=15) # Fist 15 Choice

## Data Pre Processing

In [29]:

# Transform Every Transaction to Seperate List & Gather Them into Numpy Array
transaction = lst  
transaction = np.array(lst)

te = TransactionEncoder()
te_ary = te.fit(transaction).transform(transaction)
dataset = pd.DataFrame(te_ary, columns=te.columns_)
dataset


/var/folders/ns/k85r5x0x199f1t_q2kyrlp3r0000gn/T/ipykernel_4478/2812828825.py:5: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.



,agro_industry_and_commerce,air_conditioning,art,arts_and_craftmanship,audio,auto,baby,bed_bath_table,books_general_interest,books_technical,...,pet_shop,signaling_and_security,small_appliances,small_appliances_home_oven_and_coffee,sports_leisure,stationery,tablets_printing_image,telephony,toys,watches_gifts
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
836,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
837,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
838,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [17]:
first1 = df_table["items"].head(1).values # Select Top50
dataset.loc[:,first1] # Extract Top50

,furniture_decor
0,False
1,False
2,False
3,True
4,True
...,...
835,False
836,False
837,False
838,True


In [ ]:
## Here we are using the associate learning algorithm which will be using past data to calculate support, lift and confidence. 
# We are using these values against itemsets such as audio, art etc and then using the values to predict when a person buys a thing of one category
#what other catergories have been frequently puchased together in the past

## Implementation of the Apriori Algorithm

In [39]:
frequent_itemsets = apriori(dataset, min_support=0.001, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

,support,itemsets,length
0,0.001190,(agro_industry_and_commerce),1
1,0.007143,(air_conditioning),1
2,0.008333,(art),1
3,0.002381,(arts_and_craftmanship),1
4,0.008333,(audio),1
...,...,...,...
381,0.001190,"(home_construction, furniture_decor, housewares)",3
382,0.001190,"(home_construction, furniture_decor, stationery)",3
383,0.001190,"(housewares, watches_gifts, furniture_decor)",3
384,0.001190,"(sports_leisure, pet_shop, furniture_decor)",3


In [40]:
frequent_itemsets[ (frequent_itemsets['length'] == 2) &
                   (frequent_itemsets['support'] >= 0.001) ]
frequent_itemsets.sort_values(by='support', ascending=False)

,support,itemsets,length
36,0.292857,(furniture_decor),1
7,0.273810,(bed_bath_table),1
46,0.136905,(housewares),1
6,0.119048,(baby),1
136,0.102381,"(bed_bath_table, furniture_decor)",2
...,...,...,...
197,0.001190,"(construction_tools_construction, kitchen_dini...",2
198,0.001190,"(construction_tools_construction, pet_shop)",2
199,0.001190,"(construction_tools_construction, toys)",2
200,0.001190,"(construction_tools_construction, watches_gifts)",2


In [41]:
frequent_itemsets[ (frequent_itemsets['length'] == 3) ].head()

,support,itemsets,length
358,0.00119,"(watches_gifts, audio, cool_stuff)",3
359,0.00119,"(bed_bath_table, auto, cool_stuff)",3
360,0.00119,"(auto, musical_instruments, fashion_bags_acces...",3
361,0.00119,"(baby, bed_bath_table, cool_stuff)",3
362,0.00119,"(baby, bed_bath_table, furniture_decor)",3


In [21]:
# We can create our rules by defining metric and its threshold.

# For a start, 
#      We set our metric as "Lift" to define whether antecedents & consequents are dependent our not.
#      Treshold is selected as "1.2" since it is required to have lift scores above than 1 if there is dependency.

rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0.8)
rules["antecedents_length"] = rules["antecedents"].apply(lambda x: len(x))
rules["consequents_length"] = rules["consequents"].apply(lambda x: len(x))
rules.sort_values("lift",ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedents_length,consequents_length
219,(fashio_female_clothing),(fashion_sport),0.001190,0.003571,0.001190,1.000000,280.000000,0.001186,inf,1,1
218,(fashion_sport),(fashio_female_clothing),0.003571,0.001190,0.001190,0.333333,280.000000,0.001186,1.498214,1,1
467,(food_drink),"(sports_leisure, luggage_accessories)",0.007143,0.001190,0.001190,0.166667,140.000000,0.001182,1.198571,1,2
395,(fashion_childrens_clothes),"(baby, fashion_bags_accessories)",0.002381,0.003571,0.001190,0.500000,140.000000,0.001182,1.992857,1,2
394,"(baby, fashion_bags_accessories)",(fashion_childrens_clothes),0.003571,0.002381,0.001190,0.333333,140.000000,0.001182,1.496429,2,1
...,...,...,...,...,...,...,...,...,...,...,...
77,(bed_bath_table),(costruction_tools_garden),0.273810,0.010714,0.002381,0.008696,0.811594,-0.000553,0.997964,1,1
481,"(watches_gifts, furniture_decor)",(housewares),0.010714,0.136905,0.001190,0.111111,0.811594,-0.000276,0.970982,2,1
76,(costruction_tools_garden),(bed_bath_table),0.010714,0.273810,0.002381,0.222222,0.811594,-0.000553,0.933673,1,1
131,(office_furniture),(computers_accessories),0.021429,0.069048,0.001190,0.055556,0.804598,-0.000289,0.985714,1,1


In [22]:
rules.sort_values("confidence",ascending=False)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedents_length,consequents_length
0,(agro_industry_and_commerce),(furniture_decor),0.001190,0.292857,0.00119,1.000000,3.414634,0.000842,inf,1,1
438,"(home_construction, computers_accessories)",(cool_stuff),0.001190,0.089286,0.00119,1.000000,11.200000,0.001084,inf,2,1
464,"(luggage_accessories, food_drink)",(sports_leisure),0.001190,0.096429,0.00119,1.000000,10.370370,0.001076,inf,2,1
427,"(bed_bath_table, perfumery)",(market_place),0.001190,0.014286,0.00119,1.000000,70.000000,0.001173,inf,2,1
462,"(sports_leisure, luggage_accessories)",(food_drink),0.001190,0.007143,0.00119,1.000000,140.000000,0.001182,inf,2,1
...,...,...,...,...,...,...,...,...,...,...,...
423,(furniture_decor),"(home_construction, bed_bath_table)",0.292857,0.001190,0.00119,0.004065,3.414634,0.000842,1.002886,1,2
473,(furniture_decor),"(home_construction, housewares)",0.292857,0.002381,0.00119,0.004065,1.707317,0.000493,1.001691,1,2
251,(furniture_decor),(flowers),0.292857,0.002381,0.00119,0.004065,1.707317,0.000493,1.001691,1,1
1,(furniture_decor),(agro_industry_and_commerce),0.292857,0.001190,0.00119,0.004065,3.414634,0.000842,1.002886,1,1


In [23]:
rules[~rules["consequents"].str.contains("mineral water", regex=False) & 
      ~rules["antecedents"].str.contains("mineral water", regex=False)].sort_values("confidence", ascending=False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedents_length,consequents_length
0,(agro_industry_and_commerce),(furniture_decor),0.001190,0.292857,0.001190,1.0,3.414634,0.000842,inf,1,1
438,"(home_construction, computers_accessories)",(cool_stuff),0.001190,0.089286,0.001190,1.0,11.200000,0.001084,inf,2,1
464,"(luggage_accessories, food_drink)",(sports_leisure),0.001190,0.096429,0.001190,1.0,10.370370,0.001076,inf,2,1
427,"(bed_bath_table, perfumery)",(market_place),0.001190,0.014286,0.001190,1.0,70.000000,0.001173,inf,2,1
462,"(sports_leisure, luggage_accessories)",(food_drink),0.001190,0.007143,0.001190,1.0,140.000000,0.001182,inf,2,1
23,(audio),(watches_gifts),0.008333,0.058333,0.008333,1.0,17.142857,0.007847,inf,1,1
91,(party_supplies),(bed_bath_table),0.001190,0.273810,0.001190,1.0,3.652174,0.000865,inf,1,1
25,(christmas_supplies),(auto),0.001190,0.044048,0.001190,1.0,22.702703,0.001138,inf,1,1
234,(fashion_male_clothing),(luggage_accessories),0.001190,0.020238,0.001190,1.0,49.411765,0.001166,inf,1,1
457,"(fashion_sport, watches_gifts)",(sports_leisure),0.001190,0.096429,0.001190,1.0,10.370370,0.001076,inf,2,1


rules[rules["antecedents"].str.contains("toys", regex=False) & rules["antecedents_length"] == 1].sort_values("confidence", ascending=False).head(10)

In [ ]:
## For this example we are checking our values against the itemset toys, which is when we buy toys
# In this example it can be visualised from the values of confidence, that when toys are purchased,
# items in categories baby, cool_stuff and sports leaisure are the top three most frequently brought together,
# Once we get this info, products from top three categories will be displayed to the user with the highest confidence when he is about to checkout.